Below are parameters of the experiments reported in [KLNO24](https://eprint.iacr.org/2024/1972).

In [1]:
def largest_a_below_binom_sum(n, threshold_power):
    """
    Returns the largest a such that sum_{i=0}^a binomial(n, i) < 2^threshold_power

    Parameters:
    - n: total number of bits
    - threshold_power: the exponent k such that the threshold is 2^k

    Returns:
    - a: largest integer such that the cumulative binomial sum is less than 2^threshold_power
    """
    threshold = 2^threshold_power
    running_sum = 0
    a = 0

    while True:
        running_sum += binomial(n, a)
        if running_sum >= threshold:
            return a - 1
        a += 1

# Example usage:
assert largest_a_below_binom_sum(256, 128) == 29

In [2]:
from rok_estimator.rok_estimator import *


def RelationRPWithKJL(ratio = 18, height = 105, secpar=110):
    class JL:
        kjl = ratio
        njl = height
        mjl = kjl * njl
        alpha_inf = largest_a_below_binom_sum(njl,secpar)
        
        def get_delta(self):
            return 2^(-secpar)
    
        def get_right_bound(self):
            return sqrt(self.mjl * self.njl)
            
        def get_left_bound(self):
            return self.alpha_inf / (2 * sqrt(self.mjl))
    
        def get_right_bound_inf(self):
            return self.mjl
            
        def get_left_bound_inf(self):
            return self.alpha_inf / (2)
            

    # class HJL: 
    #     kjl = ratio
    #     njl = 120
    #     mjl = kjl * njl
    #     alpha_inf = 42 
    
    #     def get_delta(self):
    #         return 2^(-110)
    
    #     def get_right_bound(self):
    #         return sqrt(207)
            
    #     def get_left_bound(self):
    #         return sqrt(7)
    
    #     def get_right_bound_inf(self):
    #         return self.mjl
            
    #     def get_left_bound_inf(self):
    #         return self.alpha_inf / (2)
    
    
    
    
    jl = JL()
    
    print("Limits of structure: ", jl.mjl)
    
    
    g_first_branch = None
    g_second_branch = None
    
    g_second_branch_ext_2_norm = None
    g_second_branch_ext_inf_norm = None
    
    class Relation_RP(Relation):
        def execute(self, op, **kwargs):
            match op:
                case "srp":
                    return self.pi_srp(**kwargs)
                case "to-first":
                    return self.pi_to_first(**kwargs)
                case "join":
                    return self.pi_join(**kwargs)
                case "norm-poly":
                    return self.pi_norm_poly()
                case "urp":
                    return self.pi_urp()
                case "usplit":
                    return self.pi_usplit(**kwargs)
                case _:
                    return super().execute(op, **kwargs)
                
        def pi_norm_poly(self):
            comm = (3 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.size_Rq()  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            # comm = (2 * self.rep) * self.ring.size_Rq() + (1 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            snd_err = 3 * ceil(log(self.wdim, 2))/(2**(self.ring.log_q * self.ring.residue_deg)) # TODO: Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the soundness error.
            rel_params = {
                # "ring": self.ring,
                # "trivial": self.trivial,
                "op_name": "norm-poly",
                "n_compress": self.n_compress + 2,
                # "n_commit": self.n_commit,
                "n_rel": self.n_rel + 2,
                # "wdim": self.wdim,
                # "rep": self.rep,
                # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
                # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
                # "log_beta_wit_inf": self.log_beta_wit_inf
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,           
                "log_beta_ext_2_func" : lambda x, _ : self.log_beta_wit_2, # perfect extraction
                "log_beta_ext_inf_func" : lambda x, _ : oo # extraction of ell_2-norm is perfect, then bound ell_inf-norm by norm conversion
            }      
            return replace(self, **rel_params)
                
        def pi_srp(self):
            global g_first_branch 
            rel_params = {
                "n_compress": self.n_commit + 1,
                "n_rel": self.n_rel + 1,
                "log_slack_2_func": lambda x: x,
                "log_slack_inf_func": lambda x: x, 
            }
    
            g_first_branch = replace(deepcopy(self), **rel_params)
            comm = (self.n_commit + 1) * self.ring.size_Rq()  # for the new comm
            snd_err = self.wdim * (1 / 2**(self.ring.log_q * self.ring.residue_deg) + jl.get_delta() * euler_phi(self.ring.f) / jl.njl)
            global g_second_branch_ext_2_norm
            global g_second_branch_ext_inf_norm
    
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                return g_second_branch_ext_2_norm + log(1 / jl.get_left_bound(), 2) + log(sqrt(radical(self.ring.f)),2)
                
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm 
                return g_second_branch_ext_inf_norm + log(1 / jl.get_left_bound_inf(), 2)
                
            rel_params = {
                "op_name": "srp 2 rel",
                "n_compress": self.n_commit + 1,
                "n_rel": 1,
                "wdim": ZZ(ceil(self.wdim / jl.kjl * self.rep)),
                "rep": 1,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,    
                "log_beta_wit_2": self.log_beta_wit_2 + log(jl.get_right_bound(), 2) + log(sqrt(self.rep),2),
                "log_beta_wit_inf":self.log_beta_wit_inf + log(jl.get_right_bound_inf(), 2) + log(sqrt(self.rep),2), # can we say sth about infinity - norm?
                "log_beta_ext_2_func": log_beta_ext_2_func, # we extract norm from the second branch (relax it)
                "log_beta_ext_inf_func" : log_beta_ext_inf_func,  # can we say sth about infinity - norm?
                "log_slack_2_func": lambda x: 0,
                "log_slack_inf_func": lambda x: 0,
            }     
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)

        def pi_urp(self):
            comm = self.rep * self.ring.size_Rq() + (jl.njl * self.rep*self.ring.log_q) * 3 # doing intermediate batching 3 times is enough for all tests we consider and selected q
            snd_err = jl.get_delta() + 2^(-256) #
            # 2^(-256) is upper-bouding due to python insufficient precision (jl.njl / (self.rep* 2**self.ring.log_q))**4  + (jl.njl / (2 * self.rep* 2**self.ring.log_q ** 2))**2 + (jl.njl / (4 * self.rep * 2**self.ring.log_q)** 4) # 
            rel_params = {
                "op_name": "uns-rp",
                "n_compress": self.n_compress + 1,
                "n_rel": self.n_rel + 1,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,  
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,    
                "log_beta_ext_2_func": lambda x, _: self.log_beta_wit_2 + log(self.wdim*euler_phi(self.ring.f)*sqrt(jl.njl * radical(self.ring.f) / jl.alpha_inf) ,2), # we extract norm from the second branch (relax it)
                "log_beta_ext_inf_func": lambda x, _: oo,  # can we say sth about infinity - norm?
                "log_slack_2_func": lambda x: 0,
                "log_slack_inf_func": lambda x: 0,
            }     
                
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)

        def pi_usplit(self, d):
            candidate = self.pi_split(d)

            comm = comm = self.rep * self.ring.size_Rq() * (d**2 - 1)
            
            rel_params = {
                "op_name": "u-split",
                "n_compress": self.n_compress + d,
                "n_rel": self.n_rel + d,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,  
            }     
                
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)
    
    
        def pi_to_first(self):
            global g_second_branch 
            g_second_branch = deepcopy(self)
            
            rel_params = {
                "op_name": "    1 rel",
                "n_rel": 1,
                "log_beta_ext_2_func" : lambda x, _ : x, # perfect extraction
                "log_beta_ext_inf_func" : lambda x, _ : x, # perfect extraction
                "acc_comm": g_second_branch.acc_comm,
                "comm": 0,
                "snd_err" : g_second_branch.snd_err,
                "acc_snd_err" : g_second_branch.acc_snd_err,    
                # "log_slack_2_func" : lambda x: 0, # perfect extraction
                # "log_slack_inf_func" : lambda x: 0 # perfect extraction
            }      
            global g_first_branch
            return replace(g_first_branch, **rel_params)
            
        def pi_join(self):
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                g_second_branch_ext_2_norm = x
                return x
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm
                g_second_branch_ext_inf_norm = x
                return x
    
            global g_second_branch
    
            assert g_second_branch.wdim <= self.wdim
            assert g_second_branch.rep == 1
            assert self.rep == 1

            comm = (self.n_rel + g_second_branch.n_rel) * self.ring.size_Rq()  # for the new comm
            
            rel_params = {
                "op_name": "join",
                "rep": 2,
                "n_rel": self.n_rel + g_second_branch.n_rel,
                "n_compress": self.n_compress + g_second_branch.n_compress,
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,  
                "log_beta_wit_2": max(self.log_beta_wit_2, g_second_branch.log_beta_wit_2),
                "log_beta_wit_inf": max(self.log_beta_wit_inf, g_second_branch.log_beta_wit_inf),
                "log_beta_ext_2_func": log_beta_ext_2_func, # perfect extraction, but we keep track of what was there
                "log_beta_ext_inf_func" : log_beta_ext_inf_func, # perfect extraction, but we keep track of what was there
                "log_slack_2_func": lambda x: 0, # assume no slack
                "log_slack_inf_func": lambda x: 0,
            }      
            return replace(self, **rel_params)
    return Relation_RP
        

def challenge_set(max_v = 100,  f = 60):
    K = CyclotomicField(f)
    phi = euler_phi(f)
    R = K.ring_of_integers()
    
    
    return ChallengeSet(cardinality = max_v^phi, gamma_2 = max_v * sqrt(phi) / 2, theta_2 = max_v * sqrt(phi), gamma_inf = max_v / 2, theta_inf = max_v)

In [3]:
# A 80

log_beta_sis_2 = 37.8

ring_params = {
    "f": 60,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": 45,
    "secpar_target": 80,
    "kappa_target": 80
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 34/  0 )         | 7687.50 MB |      (0.000000 B | 0.000000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 31 | 34/  0 )        |     ( 14 | 34/  0 )         | 7687.50 MB |      (105.469 KB | 105.469 KB)      |         (2^-85  | 2^-85 )         
 bdecomp    |   10747904 | 100 |    ( 19 | 23/  0 )        |     (  3 | 23/  0 )         | 8200.00 MB |      (329.590 KB | 435.059 KB)      |         (2^-oo  | 2^-85 )         
 norm       |   10747904 | 109 |    ( 19 | 37/  0 )        |     (  3 | 37/  0 )         | 8938.00 MB |      (68.2910 KB | 503.350 KB)      |         (2^-86  | 2^-84 )         
 batch      |   10747904 | 109 |    ( 19 | 37/  0 )        |     (  3 | 37/  0 )         | 8938.00 MB 

In [4]:
# A NEW

log_beta_sis_2 = 53.3
ring_params = {
    "f": 60,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": 59,
    "residue_deg": 5,
    "secpar_target": 80,
    "kappa_target": 80
    
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 14 + uloop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 106, 106))
sim.ring.C = challenge_set(100)


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  1908
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^59, beta_sis_2: 2^53.3000000000000
    SIS module rank n_sis: 75, target SIS security: 80, resulting SIS security: 80
    residue degree: 5, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: None
    |R_q| = 118.000 B, |R_q^(n_sis)| = 8.64258 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 33/  0 )        |     ( 14 | 33/  0 )         | 7687.50 MB |      (0.000000 B | 0.000000 B)      |         (2^-oo  | 2^-oo ) 

In [5]:
# A 128


log_beta_sis_2 = 38.6

ring_params = {
    "f": 60,
    "log_beta_sis_2": 38.6,
    "log_q": 45,
    "secpar_target": 128,
    "kappa_target": 128
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 34/  0 )         | 7687.50 MB |      (0.000000 B | 0.000000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 31 | 34/  0 )        |     ( 14 | 34/  0 )         | 7687.50 MB |      (170.859 KB | 170.859 KB)      |         (2^-130 | 2^-130)         
 bdecomp    |   10747904 | 100 |    ( 19 | 23/  0 )        |     (  3 | 23/  0 )         | 8200.00 MB |      (533.936 KB | 704.795 KB)      |         (2^-oo  | 2^-130)         
 norm       |   10747904 | 109 |    ( 19 | 38/  0 )        |     (  3 | 37/  0 )         | 8938.00 MB |      (92.8125 KB | 797.607 KB)      |         (2^-131 | 2^-129)         
 batch      |   10747904 | 109 |    ( 19 | 38/  0 )        |     (  3 | 37/  0 )         | 8938.00 MB 

In [11]:
# A NEW 128

log_beta_sis_2 = 56.4
ring_params = {
    "f": 60,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": 60,
    "residue_deg": 6,
    "secpar_target": 128,
    "kappa_target": 128
    
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 14 + uloop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 154, 154))
sim.ring.C = challenge_set(400)


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2772
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^60, beta_sis_2: 2^56.4000000000000
    SIS module rank n_sis: 130, target SIS security: 128, resulting SIS security: 129
    residue degree: 6, target Schwartz-Zippel security: 128, resulting Schwartz-Zippel security: None
    |R_q| = 120.000 B, |R_q^(n_sis)| = 15.2344 KB
 
Challenge set parameters:
    cardinality: 429496729600000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^10
    inverse ell_2 expansion factor theta_2: 2^11
    forward ell_inf expansion factor gamma_inf: 2^8
    inverse ell_inf expansion factor theta_inf: 2^9
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 33/  0 )        |     ( 14 | 33/  0 )         | 7687.50 MB |      (0.000000 B | 0.000000 B)      |         (2^

In [13]:
# A 196

log_beta_sis_2 = 44.9
ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": log_beta_sis_2 + 3,
    "secpar_target": 196,
    "kappa_target": 196,
    "residue_deg": 5
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 35/  0 )         | 7687.50 MB |      (0.000000 B | 0.000000 B)      |         (2^-oo  | 2^-oo )         
 split      |    5373952 |  25 |    ( 31 | 35/  0 )        |     ( 14 | 35/  0 )         | 7687.50 MB |      (336.797 KB | 336.797 KB)      |         (2^-234 | 2^-234)         
 bdecomp    |    5373952 | 100 |    ( 20 | 23/  0 )        |     (  3 | 23/  0 )         | 8200.00 MB |      (1.02782 MB | 1.35673 MB)      |         (2^-oo  | 2^-234)         
 norm       |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938.00 MB |      (187.484 KB | 1.53982 MB)      |         (2^-236 | 2^-234)         
 batch      |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938.00 MB 

# A NEW 196

log_beta_sis_2 =  56

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": 62,
    "residue_deg": 8,
    "secpar_target": 196,
    "kappa_target": 196
    
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 14 + uloop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 226, 226))
sim.ring.C = challenge_set(83,ring_params["f"])


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

In [9]:
# A 256

log_beta_sis_2 =  45.5
ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": log_beta_sis_2 + 3,
    "secpar_target": 256,
    "kappa_target": 256
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 35/  0 )         | 7687.50 MB |      (0.000000 B | 0.000000 B)      |         (2^-oo  | 2^-oo )         
 split      |    5373952 |  25 |    ( 31 | 35/  0 )        |     ( 14 | 35/  0 )         | 7687.50 MB |      (436.500 KB | 436.500 KB)      |         (2^-286 | 2^-286)         
 bdecomp    |    5373952 | 100 |    ( 20 | 23/  0 )        |     (  3 | 23/  0 )         | 8200.00 MB |      (1.33209 MB | 1.75836 MB)      |         (2^-oo  | 2^-286)         
 norm       |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938.00 MB |      (225.639 KB | 1.97871 MB)      |         (2^-287 | 2^-285)         
 batch      |    5373952 | 109 |    ( 20 | 43/  0 )        |     (  3 | 43/  0 )         | 8938.00 MB 

In [19]:
### A NEW 256

log_beta_sis_2 =  59.6

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": log_beta_sis_2,
    "log_q": log_beta_sis_2 + 3,
    "secpar_target": 256,
    "kappa_target": 256,
    "residue_deg": 5
}

rel_params = {
    "wdim": 2**17 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])


uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop(2) + loop(2) * 13 + uloop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 280, 280))
sim.ring.C = challenge_set(639, ring_params["f"])


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  5040
Ring parameters:
    conductor f: 120, degree phi: 32, modulus q: 2^62.6000000000000, beta_sis_2: 2^59.6000000000000
    SIS module rank n_sis: 127, target SIS security: 256, resulting SIS security: 256
    residue degree: 5, target Schwartz-Zippel security: 256, resulting Schwartz-Zippel security: None
    |R_q| = 250.400 B, |R_q^(n_sis)| = 31.0555 KB
 
Challenge set parameters:
    cardinality: 597073039182879857989553168123669664326517795218001082090332702878587253341837810932101121
    forward ell_2 expansion factor gamma_2: 2^11
    inverse ell_2 expansion factor theta_2: 2^12
    forward ell_inf expansion factor gamma_inf: 2^9
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  134348800 |   1 |    ( 33 | 34/  0 )        |     ( 14 | 34/  0 )         